In [6]:
from dotenv import load_dotenv

load_dotenv()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
)

# Basic Chatbot

In [4]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)



def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)
graph_builder.set_entry_point("chatbot")
graph_builder.set_finish_point("chatbot")
graph = graph_builder.compile()

In [5]:
graph.invoke({"messages": [("user", "hi!")]})

{'messages': [HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='32895d4b-6682-478f-8220-2985dfaa1f54'),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'stop', 'logprobs': None}, id='run-fb113b8a-c4f8-419c-b94c-bf9ef3840db0-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

# Adding tools

In [7]:
from typing import Annotated

from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


tool = TavilySearchResults(max_results=2)
tools = [tool]
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile()

In [8]:
graph.invoke({"messages": [("user", "quien es linus tolvard")]})

{'messages': [HumanMessage(content='quien es linus tolvard', additional_kwargs={}, response_metadata={}, id='47167559-44e0-47fe-81f4-6dac85ca776f'),
  AIMessage(content='Linus Torvalds es un ingeniero de software finlandés-estadounidense, conocido principalmente por haber creado el núcleo del sistema operativo Linux. Nació el 28 de diciembre de 1969 en Helsinki, Finlandia. Torvalds comenzó el desarrollo de Linux en 1991 como un proyecto personal mientras estudiaba en la Universidad de Helsinki. Desde entonces, Linux ha crecido hasta convertirse en uno de los sistemas operativos más importantes del mundo, utilizado en servidores, dispositivos móviles, supercomputadoras y más.\n\nAdemás de su trabajo en Linux, Torvalds también es conocido por haber creado el sistema de control de versiones Git, que es ampliamente utilizado en el desarrollo de software para gestionar el código fuente de proyectos. Su contribución al software libre y de código abierto ha tenido un impacto significativo en 

# Adding memory

In [9]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

graph = graph_builder.compile(checkpointer=memory)

In [11]:
config = {"configurable": {"thread_id": "1"}}

In [12]:
graph.invoke({"messages": [("user", "hola soy james")]}, config=config)

{'messages': [HumanMessage(content='hola soy james', additional_kwargs={}, response_metadata={}, id='17fc9c78-7f02-439f-98f1-dab5b011af7c'),
  AIMessage(content='¡Hola James! ¿En qué puedo ayudarte hoy?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 82, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_523b9b6e5f', 'finish_reason': 'stop', 'logprobs': None}, id='run-f61eeffe-42b2-43e1-bafe-12327e7e0090-0', usage_metadata={'input_tokens': 82, 'output_tokens': 13, 'total_tokens': 95, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

In [13]:
graph.invoke({"messages": [("user", "hola, quien soy?")]}, config=config)

{'messages': [HumanMessage(content='hola soy james', additional_kwargs={}, response_metadata={}, id='17fc9c78-7f02-439f-98f1-dab5b011af7c'),
  AIMessage(content='¡Hola James! ¿En qué puedo ayudarte hoy?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 82, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_523b9b6e5f', 'finish_reason': 'stop', 'logprobs': None}, id='run-f61eeffe-42b2-43e1-bafe-12327e7e0090-0', usage_metadata={'input_tokens': 82, 'output_tokens': 13, 'total_tokens': 95, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content='hola, quien soy?', additional_kwargs={}, response_metadata={}, id='5d5877